In [1]:
import cv2

# Open default camera (0)
cap = cv2.VideoCapture(0)

# Define video codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' for .mp4 on macOS
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480), isColor=False)

# Define kernel for morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # 5x5 rectangular kernel

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply grayscale filter (RGB to Grayscale)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Otsu's thresholding
    _, otsu = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Apply binary thresholding
    _, binary = cv2.threshold(otsu, 127, 255, cv2.THRESH_BINARY)

    # Apply morphological opening (erosion followed by dilation)
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    # Apply morphological closing (dilation followed by erosion)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    # Apply Canny edge detection
    edges = cv2.Canny(closing, 100, 200)

    # Convert edges to 3 channels to match the original frame
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Convert the original frame to grayscale for stitching consistency
    original_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Blend the original frame and the edges using alpha blending
    alpha = 0.6  # Transparency factor for the original frame
    beta = 1 - alpha  # Transparency factor for the edges
    overlapped = cv2.addWeighted(frame, alpha, edges_colored, beta, 0)



    # Show the video
    cv2.imshow('Gray Filtered Video', overlapped)

    # Write the frame
    out.write(overlapped)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 